In [2]:
library(tidyverse)
library(lme4)
library(lmerTest)
library(ggpubr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack



Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step


Registered S3 methods overwritten by 'car':
  method                          from
  influence.merMod                lme4
  cooks.distance.influence.merMod lme4
  dfbeta.influence.merMod         lme4
  dfbetas.influence.merMod        lme4



In [3]:
tables_path = "/home/marc/ba-thesis/input/tables"

mean_amplitudes_df = readr::read_csv("../data/mean_amplitudes.csv")
mean_amplitudes_df = mean_amplitudes_df %>% mutate(Participant = factor(Participant), SOA = factor(SOA), Electrode = factor(Electrode), Condition = factor(Condition), StimulusType = factor(StimulusType))
mean_amplitudes_df


── Column specification ────────────────────────────────────────────────────────
cols(
  SOA = col_double(),
  Participant = col_character(),
  Condition = col_character(),
  StimulusType = col_character(),
  MeanAmplitude = col_double(),
  Electrode = col_character()
)




SOA,Participant,Condition,StimulusType,MeanAmplitude,Electrode
<fct>,<fct>,<fct>,<fct>,<dbl>,<fct>
100,100_0,random,B,1.458006e-06,FZ
100,100_1,random,B,2.571833e-08,FZ
100,100_2,random,B,1.964786e-07,FZ
100,100_3,random,B,-4.050463e-07,FZ
100,100_4,random,B,-1.272691e-06,FZ
100,100_5,random,B,2.706422e-07,FZ
100,100_6,random,B,-4.913439e-07,FZ
100,100_7,random,B,1.300596e-06,FZ
100,100_8,random,B,3.499203e-06,FZ


In [3]:


mean_amplitudes_150 = mean_amplitudes_df %>% filter(SOA == "150" & Electrode == "fronto_pooled") %>% droplevels()

model1 = lmer(MeanAmplitude ~ Condition * StimulusType + (1|Participant) + (1|Participant:Condition) + (1|Participant:StimulusType), data = mean_amplitudes_150)
model2 = aov(MeanAmplitude~Condition*StimulusType+Error(Participant/(Condition*StimulusType)), data=mean_amplitudes_150)
model3 = ez::ezANOVA(mean_amplitudes_150, dv = MeanAmplitude,  wid = Participant, within = c(Condition, StimulusType), return_aov=T)
model4 = nlme::lme(MeanAmplitude ~ Condition * StimulusType, random=~1|Participant, correlation=nlme::corCompSymm(form=~1|Participant), data=mean_amplitudes_150)
model5 = lmer(MeanAmplitude ~ (1|Participant) + (1|Participant:Condition) + (1|Participant:StimulusType), data = mean_amplitudes_150)

car::Anova(model1) # car on lmer
anova(model1) # lmerTest
summary(model2) # aov (always type I)
model3$ANOVA # ez
anova(model4)


#ggplot(data=mean_amplitudes_100) +
#    geom_boxplot(aes(x = Condition, y = MeanAmplitude, color = StimulusType))

boundary (singular) fit: see ?isSingular



,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>
Condition,4.797969,1,2.849330e-02
StimulusType,56.544224,1,5.494857e-14
Condition:StimulusType,4.197179,1,4.049129e-02


,Sum Sq,Mean Sq,NumDF,DenDF,F value,Pr(>F)
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
Condition,5.818108e-13,5.818108e-13,1,0.2704110,4.797969,0.5739043
StimulusType,6.856659e-12,6.856659e-12,1,0.2703858,56.544224,0.4127375
Condition:StimulusType,5.089578e-13,5.089578e-13,1,0.2738294,4.197179,0.5811995



Error: Participant
          Df    Sum Sq   Mean Sq F value Pr(>F)
Residuals 22 2.082e-11 9.463e-13               

Error: Participant:Condition
          Df    Sum Sq   Mean Sq F value Pr(>F)  
Condition  1 1.433e-12 1.433e-12   4.798 0.0394 *
Residuals 22 6.568e-12 2.986e-13                 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Error: Participant:StimulusType
             Df    Sum Sq   Mean Sq F value   Pr(>F)    
StimulusType  1 2.304e-11 2.304e-11   56.54 1.62e-07 ***
Residuals    22 8.963e-12 4.070e-13                     
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Error: Participant:Condition:StimulusType
                       Df    Sum Sq   Mean Sq F value Pr(>F)  
Condition:StimulusType  1 5.090e-13 5.090e-13   4.197 0.0526 .
Residuals              22 2.668e-12 1.213e-13                 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

,Effect,DFn,DFd,F,p,p<.05,ges
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
2,Condition,1,22,4.797994,3.938039e-02,*,0.03541417
3,StimulusType,1,22,56.544332,1.624601e-07,*,0.37122501
4,Condition:StimulusType,1,22,4.197170,5.260654e-02,,0.01287632


,numDF,denDF,F-value,p-value
,<int>,<dbl>,<dbl>,<dbl>
(Intercept),1,66,15.771762,1.792030e-04
Condition,1,66,5.195154,2.589046e-02
StimulusType,1,66,83.541869,2.453593e-13
Condition:StimulusType,1,66,1.845790,1.788993e-01


In [4]:
library(rstatix)
library(xtable)
library(huxtable)

variables = list()

### 100 ms x 150ms
## ANOVA with pooled electrodes
anova_00_data = mean_amplitudes_df %>% filter(Electrode == "fronto_pooled") %>% droplevels() 
anova_00_table =    anova_00_data %>% 
                        anova_test(dv = MeanAmplitude,  wid = Participant, within = c(Condition, StimulusType), between = c(SOA)) %>%
                        get_anova_table(correction = "auto")
anova_00_table$Effect = str_replace_all(anova_00_table$Effect, ":", " x ")
variables$anova_00_soa = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$, $\\eta_{gen}=%.4f$", anova_00_table$DFn[[1]], anova_00_table$DFd[[1]], anova_00_table$F[[1]], anova_00_table$p[[1]], anova_00_table$ges[[1]])
print(xtable(anova_00_table), label = "anova_00_table", include.rownames = F, tabular.environment = "tabulary", width = "\\textwidth", file = paste(tables_path, "anova_00.tex", sep="/"))

### 100 ms
## ANOVA for 100 ms with pooled electrodes (fronto x mastoids)
anova_02_100_data = mean_amplitudes_df %>% filter(SOA == "100") %>% filter(Electrode == "fronto_pooled" | Electrode == "mastoids_pooled") %>% droplevels() 

anova_02_100_table =    anova_02_100_data %>% 
                        anova_test(dv = MeanAmplitude,  wid = Participant, within = c(Condition, StimulusType, Electrode)) %>%
                        get_anova_table(correction = "auto")
anova_02_100_table$Effect = str_replace_all(anova_02_100_table$Effect, ":", " x ")
variables$anova_02_100_condition = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_100_table$DFn[[1]], anova_02_100_table$DFd[[1]], anova_02_100_table$F[[1]], anova_02_100_table$p[[1]])
variables$anova_02_100_stimulustype = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_100_table$DFn[[2]], anova_02_100_table$DFd[[2]], anova_02_100_table$F[[2]], anova_02_100_table$p[[2]])
variables$anova_02_100_electrode = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_100_table$DFn[[3]], anova_02_100_table$DFd[[3]], anova_02_100_table$F[[3]], anova_02_100_table$p[[3]])
variables$anova_02_100_condition_stimulustype = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_100_table$DFn[[4]], anova_02_100_table$DFd[[4]], anova_02_100_table$F[[4]], anova_02_100_table$p[[4]])
variables$anova_02_100_condition_electrode = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_100_table$DFn[[5]], anova_02_100_table$DFd[[5]], anova_02_100_table$F[[5]], anova_02_100_table$p[[5]])
variables$anova_02_100_stimulustype_elctrode = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_100_table$DFn[[6]], anova_02_100_table$DFd[[6]], anova_02_100_table$F[[6]], anova_02_100_table$p[[6]])
variables$anova_02_100_condition_stimulustype_electrode = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_100_table$DFn[[7]], anova_02_100_table$DFd[[7]], anova_02_100_table$F[[7]], anova_02_100_table$p[[7]])
print(xtable(anova_02_100_table), include.rownames = F, tabular.environment = "tabulary", width = "\\textwidth", file = paste(tables_path, "anova_01_100.tex", sep="/"))


### 100 ms
## ANOVA for 100 ms with pooled electrodes
anova_01_100_data = mean_amplitudes_df %>% filter(SOA == "100" & Electrode == "fronto_pooled") %>% droplevels() 
anova_01_100_table =    anova_01_100_data %>% 
                        anova_test(dv = MeanAmplitude,  wid = Participant, within = c(Condition, StimulusType)) %>%
                        get_anova_table()
anova_01_100_table$Effect = str_replace_all(anova_01_100_table$Effect, ":", " x ")
variables$anova_01_100_condition = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_01_100_table$DFn[[1]], anova_01_100_table$DFd[[1]], anova_01_100_table$F[[1]], anova_01_100_table$p[[1]])
variables$anova_01_100_stimulustype = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_01_100_table$DFn[[2]], anova_01_100_table$DFd[[2]], anova_01_100_table$F[[2]], anova_01_100_table$p[[2]])
variables$anova_01_100_condition_stimulustype = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_01_100_table$DFn[[3]], anova_01_100_table$DFd[[3]], anova_01_100_table$F[[3]], anova_01_100_table$p[[3]])
caption  = "Results of the 3-way ANOVA (condition x stimulus x electrode) for repeated measures conducted on the mean ERP-amplitudes (time window 111 - 161 ms) at electrode Fz (upper section). The significant interaction between the three factors included was further analyzed by 2-way ANOVAS (stimulus x electrode) conducted separately for the random condition (middle section) and the predictable condition (lower section)."
print(xtable(anova_01_100_table, correction = "auto", caption = caption), include.rownames = F, tabular.environment = "tabulary", width = "\\textwidth", file = paste(tables_path, "anova_02_100.tex", sep="/"))

# ## ANOVA for 100 ms with non-pooled electrodes
# anova_02_100_data = mean_amplitudes_df %>% filter(SOA == "100" & Electrode != "fronto_pooled") %>% droplevels() 
# anova_02_100_table = anova_02_100_data %>% 
#                      anova_test(dv = MeanAmplitude,  wid = Participant, within = c(Condition, StimulusType, Electrode)) %>%
#                      get_anova_table(correction = "auto")
# anova_02_100_table$Effect = str_replace_all(anova_02_100_table$Effect, ":", " x ")
# caption  = "Results from two-way ANOVA for 100 ms with non-pooled electrodes"
# print(xtable(anova_02_100_table, caption=caption), include.rownames = F, tabular.environment = "tabulary", booktabs = TRUE, width = "\\textwidth", file = paste(tables_path, "anova_02_100.tex", sep="/"))

# Post-hoc
anova_02_100_posthoc = anova_02_100_data %>% 
                       group_by(Participant, Electrode) %>% summarize(MeanAmplitude = mean(MeanAmplitude)) %>% ungroup() %>%
                       t_test(MeanAmplitude ~ Electrode, paired=TRUE) %>%
                       rename(t = statistic) %>% select(group1, group2, df, t, p, p.adj)
print(xtable(anova_02_100_posthoc, correction = "auto"), include.rownames = F, tabular.environment = "tabulary", booktabs = TRUE, width = "\\textwidth",  file = paste(tables_path, "anova_02_100_posthoc.tex", sep="/"))

### 150 ms
## ANOVA for 150 ms with pooled electrodes (fronto x mastoids)
anova_02_150_data = mean_amplitudes_df %>% filter(SOA == "150" & (Electrode == "fronto_pooled" | Electrode == "mastoids_pooled")) %>% droplevels() 

anova_02_150_table =    anova_02_150_data %>% 
                        anova_test(dv = MeanAmplitude,  wid = Participant, within = c(Condition, StimulusType, Electrode)) %>%
                        get_anova_table(correction = "auto")
anova_02_150_table$Effect = str_replace_all(anova_02_150_table$Effect, ":", " x ")
variables$anova_02_150_condition = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_150_table$DFn[[1]], anova_02_150_table$DFd[[1]], anova_02_150_table$F[[1]], anova_02_150_table$p[[1]])
variables$anova_02_150_stimulustype = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_150_table$DFn[[2]], anova_02_150_table$DFd[[2]], anova_02_150_table$F[[2]], anova_02_150_table$p[[2]])
variables$anova_02_150_electrode = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_150_table$DFn[[3]], anova_02_150_table$DFd[[3]], anova_02_150_table$F[[3]], anova_02_150_table$p[[3]])
variables$anova_02_150_condition_stimulustype = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_150_table$DFn[[4]], anova_02_150_table$DFd[[4]], anova_02_150_table$F[[4]], anova_02_150_table$p[[4]])
variables$anova_02_150_condition_electrode = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_150_table$DFn[[5]], anova_02_150_table$DFd[[5]], anova_02_150_table$F[[5]], anova_02_150_table$p[[5]])
variables$anova_02_150_stimulustype_elctrode = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_150_table$DFn[[6]], anova_02_150_table$DFd[[6]], anova_02_150_table$F[[6]], anova_02_150_table$p[[6]])
variables$anova_02_150_condition_stimulustype_electrode = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_02_150_table$DFn[[7]], anova_02_150_table$DFd[[7]], anova_02_150_table$F[[7]], anova_02_150_table$p[[7]])
print(xtable(anova_02_150_table), include.rownames = F, tabular.environment = "tabulary", width = "\\textwidth", file = paste(tables_path, "anova_01_150.tex", sep="/"))


## ANOVA for 150 ms with pooled electrodes
anova_01_150_data = mean_amplitudes_df %>% filter(SOA == "150" & Electrode == "fronto_pooled") %>% droplevels() 
anova_01_150_table =    anova_01_150_data %>% 
                        anova_test(dv = MeanAmplitude,  wid = Participant, within = c(Condition, StimulusType)) %>%
                        get_anova_table()
anova_01_150_table$Effect = str_replace_all(anova_01_150_table$Effect, ":", " x ")
variables$anova_01_150_condition = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_01_150_table$DFn[[1]], anova_01_150_table$DFd[[1]], anova_01_150_table$F[[1]], anova_01_150_table$p[[1]])
variables$anova_01_150_stimulustype = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_01_150_table$DFn[[2]], anova_01_150_table$DFd[[2]], anova_01_150_table$F[[2]], anova_01_150_table$p[[2]])
variables$anova_01_150_condition_stimulustype = sprintf("$F(%.0f,%.0f) = %.2f$, $p = %.4f$", anova_01_150_table$DFn[[3]], anova_01_150_table$DFd[[3]], anova_01_150_table$F[[3]], anova_01_150_table$p[[3]])
print(xtable(anova_01_150_table, correction = "auto"), include.rownames = F, tabular.environment = "tabulary", width = "\\textwidth", file = paste(tables_path, "anova_01_150.tex", sep="/"))



# ## ANOVA for 150 ms with non-pooled electrodes
# anova_02_150_data = mean_amplitudes_df %>% filter(SOA == "150" & Electrode != "fronto_pooled") %>% droplevels() 
# anova_02_150_table = anova_02_150_data %>% 
#                      anova_test(dv = MeanAmplitude,  wid = Participant, within = c(Condition, StimulusType, Electrode)) %>%
#                      get_anova_table(correction = "auto")
# anova_02_150_table$Effect = str_replace_all(anova_02_150_table$Effect, ":", " x ")
# print(xtable(anova_02_150_table), include.rownames = F, tabular.environment = "tabulary", booktabs = TRUE, width = "\\textwidth", file = paste(tables_path, "anova_02_150.tex", sep="/"))

# Post-hoc
anova_02_150_posthoc = anova_02_150_data %>% 
                       group_by(Participant, Electrode) %>% summarize(MeanAmplitude = mean(MeanAmplitude)) %>% ungroup() %>%
                       t_test(MeanAmplitude ~ Electrode, paired=TRUE) %>%
                       rename(t = statistic) %>% select(group1, group2, df, t, p, p.adj)
print(xtable(anova_02_150_posthoc, correction = "auto"), include.rownames = F, tabular.environment = "tabulary", booktabs = TRUE, width = "\\textwidth",  file = paste(tables_path, "anova_02_150_posthoc.tex", sep="/"))

## Create more complex tables

anova_02_full_caption  = "Results of the 3-way ANOVA (condition x stimulus x electrode) for repeated measures conducted on the mean ERP-amplitudes (time window 111 - 161 ms) at electrode Fz (upper section). The significant interaction between the three factors included was further analyzed by 2-way ANOVAS (stimulus x electrode) conducted separately for the random condition (middle section) and the predictable condition (lower section)."

anova_02_full_table = rbind(anova_02_100_table %>% add_column(SOA = paste("100", "ms"), .before = 1), anova_02_150_table %>% add_column(SOA = paste("150", "ms"), .before =1)) %>%
                        hux() %>%
                        merge_repeated_rows(col=1) %>%
                        set_rotation(col=1, value=90) %>%
                        set_valign(col=1, value="middle") %>%
                        set_header_rows(1, TRUE) %>%
                        set_contents(1,1, value = "") %>%

                        set_caption(anova_02_full_caption) %>%

                        set_bottom_padding(row=8, value=8) %>%
                        set_top_padding(row=9, value=8) %>%

                        set_bottom_padding(row=8, value=15) %>%
                        set_top_padding(row=9, value=15) %>%
                        theme_article()

write(to_latex(anova_02_full_table), paste(tables_path, "anova_02_full.tex", sep="/"))

print_md(anova_02_full_table)

# Write Vars
yaml::write_yaml(variables, "/home/marc/ba-thesis/input/vars.yaml")




variables


Attaching package: ‘rstatix’


The following object is masked from ‘package:stats’:

    filter



Attaching package: ‘huxtable’


The following objects are masked from ‘package:xtable’:

    align, align<-, caption, caption<-, label, label<-, sanitize


The following object is masked from ‘package:ggpubr’:

    font


The following object is masked from ‘package:dplyr’:

    add_rownames


The following object is masked from ‘package:ggplot2’:

    theme_grey


`summarise()` regrouping output by 'Participant' (override with `.groups` argument)



ERROR: Error: Can't subset columns that don't exist.
[31m✖[39m Column `p.adj` doesn't exist.


In [13]:
## ANOVA Plot
library(ggthemes)

plot = ggplot(anova_02_100_data, aes(y=MeanAmplitude, x=StimulusType, color=Electrode)) +
    facet_wrap(~ Condition) +
    stat_summary(fun.data = mean_se, geom = "errorbar") +
    stat_summary(aes(group=Electrode), fun.y=mean, geom="line") +
    geom_rangeframe(color="black") +
    theme_tufte() +
    theme(text=element_text(size=8,  family="Sans")) 

ggsave("/home/marc/ba-thesis/input/figures/fig9.pdf", plot, width = 6.25, height = 2.5, units = "in")
ggsave("/home/marc/ba-thesis/input/figures/fig9.png", plot, width = 6.25, height = 2.5, units = "in")


Warning message:
“`fun.y` is deprecated. Use `fun` instead.”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Sans' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Sans' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Sans' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Sans' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Sans' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Sans' not found in PostScript font database”
Warning message in grid.Call(C_stringMetric, as.graphicsAnnot(x$label)):
“font family 'Sans' not found in PostScript font database”
Warning message

ERROR: Error in grid.Call.graphics(C_text, as.graphicsAnnot(x$label), x$x, x$y, : invalid font type


In [131]:
install.packages("ggthemes")

Installing package into ‘/home/marc/R/x86_64-pc-linux-gnu-library/4.0’
(as ‘lib’ is unspecified)



In [5]:

hx_tmp = anova_01_all_table %>% as_hux() %>% theme_basic() %>%
  #set_contents(1, 2:5, c("Length", "Width", "Length", "Width")) %>% 
  insert_row("", "100 ms", "", "", "", "",  "", "150 ms", "", "", "", "", "", after = 0) %>% 
  set_align(1,2, "center") %>%
  set_header_rows(1, TRUE) %>%

  merge_cells(1, 2:6) %>% 
  merge_cells(1, 8:13)


print_md(hx_tmp, max_width=500)

write(to_latex(hx_tmp), paste(tables_path, "anova_01_full.tex", sep="/"))

ERROR: Error in eval(lhs, parent, parent): object 'anova_01_all_table' not found


In [81]:
report_latex_dependencies()

\usepackage{array}
\usepackage{caption}
\usepackage{graphicx}
\usepackage{siunitx}
\usepackage{ulem}
\usepackage{colortbl}
\usepackage{multirow}
\usepackage{hhline}
\usepackage{calc}
\usepackage{tabularx}
\usepackage{threeparttable}
\usepackage{wrapfig}
\usepackage{adjustbox}
\usepackage{hyperref}
% These are LaTeX packages. You can install them using your LaTex management software,
% or by running `huxtable::install_latex_dependencies()` from within R.
% Other packages may be required if you use non-standard tabulars (e.g. tabulary).